Материалы для работы:
* https://habr.com/ru/articles/768562/
* https://huggingface.co/learn/audio-course/ru/chapter5/asr_models

In [1]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-medium", device=device
)

In [9]:
import os
import torchaudio
from datasets import load_dataset

In [4]:
import os
import torchaudio
from torchaudio.datasets import LIBRISPEECH
import shutil

# Step 1: Install the required library
# Run this command in your terminal
# pip install torchaudio

# Ensure the data directory exists
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

# Step 2: Download a sample from the LibriSpeech dataset
# We will use the "test-clean" subset for this example
try:
    dataset = LIBRISPEECH(data_dir, url="test-clean", download=True)
except Exception as e:
    print(f"Error downloading dataset: {e}")
    exit(1)

# Step 3: Extract the first sample from the dataset
sample = dataset[0]
waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id = sample

# Ensure the output directory exists
output_dir = "../audio"
os.makedirs(output_dir, exist_ok=True)

# Save the audio waveform as a WAV file
torchaudio.save(os.path.join(output_dir, "test_audio.wav"), waveform, sample_rate)

# Save the corresponding text to a file
with open(os.path.join(output_dir, "test_audio.txt"), "w") as f:
    f.write(utterance)


shutil.rmtree(data_dir)
print("Temporary data directory deleted successfully.")
print("Test audio WAV file and corresponding text file created successfully.")

100%|██████████| 331M/331M [00:44<00:00, 7.80MB/s] 


Test audio WAV file and corresponding text file created successfully.


In [5]:
import warnings
import difflib

# Step 1: Ignore warnings
warnings.filterwarnings("ignore")

# Step 2: Run the transcription process
result = pipe('../audio/test_audio.wav', max_new_tokens=256)

# Step 3: Print the transcribed text
transcribed_text = result["text"].lower()
print("Transcribed Text:", transcribed_text)

# Step 4: Define the original text
with open('../audio/test_audio.txt') as file:
    original_text = file.read().lower()

# Step 5: Compare the transcribed text with the original text
similarity_ratio = difflib.SequenceMatcher(None, transcribed_text, original_text).ratio()

# Step 6: Print the similarity ratio
print("Similarity Ratio:", similarity_ratio)

Transcribed Text:  he hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton-pieces to be ladled out in thick, peppered, flour-fattened sauce.
Similarity Ratio: 0.9719626168224299
